# ECON 31703 Problem Set 2 - Arjun Gopinath and Tugce Turk

In [1]:
# Standard Python Imports

import numpy as np
import pandas as pd
from numba import njit, jit
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
import statsmodels.api as sm
from scipy.stats import norm, zscore
import scipy as sp
from numpy import random, linalg
from scipy import sparse, stats
import itertools as it
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.linear_model import Lasso

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = [
    r'\usepackage{amssymb}',
    r'\usepackage{amsmath}',
    r'\usepackage{xcolor}',
    r'\renewcommand*\familydefault{\sfdefault}']
matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams['pgf.preamble']  = [
    r'\usepackage[utf8x]{inputenc}',
    r'\usepackage{amssymb}',
    r'\usepackage[T1]{fontenc}',
    r'\usepackage{amsmath}',
    r'\usepackage{sansmath}']

from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

inv, ax, norm = np.linalg.inv, np.newaxis, np.linalg.norm
randint = np.random.randint

In [39]:
N_obs = 50
N_param = 100

# Simulate data used in exercise
X, u, b = np.random.randn(N_obs, N_param), np.random.randn(N_obs, 1), np.random.randn(N_param, 1)

# Set intercept
X[:, 0] = 1.

# Random number of coefficients set to zero
n_0 = np.random.randint(0, N_param/3)
b[randint(1, N_param, n_0), :] = 0

# Set outcome variable
Y = X @ b + u

## Exercise 1 - Coordinate Gradient Descent in LASSO

### Part A: LASSO Objective Function

In [25]:
def lasso_objective(b, y, X, lmbda):
    
    """
        Function that accepts the guess for the parameter vector and LASSO penalty multipler λ, 
         and computes the LASSO objective function based on the input data.
        :param b: Parameter vector.
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param lmbda: LASSO penalty.        
        :return: Objective function evaluated using inputs.
    """

    # Return the objective function if matrix multiplication Xβ is compatible.
    try:
        obj = np.square(y - X @ b).sum() + lmbda * norm(b, ord=1)
        return obj
    except:
        print("Error: The number of covariates is not compatible with given coefficient vector.")
        return np.inf       


### Part B: Cyclic Coordinate Descent

In [80]:
def dualsol(bj, lmbda):
    """
        Function that returns the solution for a single coordinate in the Cyclic
         Coordinate Descent algorithm given the OLS coordinate estimate and the
         LASSO penalty multipler.
        :param bj: OLS estimate for coordinate j.
        :param lmbda: LASSO penalty multiplier. 
        :return: LASSO coordinate estimate.
    """
   
    if bj < - lmbda:
        return (bj + lmbda)
    elif rho >  lamda:
        return (bj - lmbda)
    else: 
        return 0


def lasso_cdg(bstart, y, X, lmbda, eps=1e-6, maxiter=1000, standardized=False):
    
    """
        Function that performs the LASSO estimation through the Cyclic Coordinate Descent algorithm.
        :param b: Initial guess for the parameter vector (may or may not include b0, which will be trimmed out if so)
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param lmbda: LASSO penalty multiplier.  
        :param eps: Norm stopping criterion.
        :param maxiter: Iteration number stopping criterion.
        :param standardized: Indicator for whether the data has been standardized.
        :return: List containing:
            - :estimate: final coefficient vector estimate, 
            - :objectives: vector containing LASSO objective function values
            - :steps: vector containing norm of difference in estimated parameter vectors
            - :status: string regarding which stopping criterion was used.
    """
    
    p, N, b_guess = bstart.size, y.size, bstart
       
    if N != X.shape[0]:      
        print("Error: Covariate matrix is incompatible with outcome variable.")
        return None
    elif p != X.shape[1]:
        print("Error: Covariate matrix is incompatible with parameter vector.")
        return None
    
    # Detect if a constant term is included
    iota = (X[:, 0] == X[:, 0].mean()).all()
    
    # Trim out constant term
    if iota:
        X, b_guess = X[:, 1:], b_guess[1:]
        p = p - 1
        
    # Standardize data if not done so
    if standardized is False:
        X_mean, y_mean = X.mean(axis=0), y.mean()
        X_std, y_std = X.std(axis=0), y.std()
        X, y = zscore(X, axis=0), zscore(y) 
        
    # LASSO objective
    lasso_obj = lambda b : lasso_objective(b, y, X, lmbda)
        
    keyDict = {"estimate", "objectives", "steps", "status"}
    output = dict([(key, []) for key in keyDict])
    
    niter, dist = 1, 1
    
    # While loop to perform LASSO minimization using two stopping criterion.
    while niter < maxiter and dist > eps:
        
        b_old = b_guess
        
        for j in np.arange(0, p):
            
            # Extract j^{th} covariate vector
            Xj = X[:,j].reshape(-1,1)
            
            # Compute OLS solution for β_j taking β_{-j} as given
            bj = X_j.T @ (y - X @ b_guess + b_guess[j] * Xj)
            
            # Update guess for j^{th} coordinate using LASSO closed form solution under CDG
            b_guess[j] = dualsol(bj, lmbda) 
            
        b0 = y_mean - np.dot(X_mean, b_guess)            
        
        output["estimate"].append(np.array([b0, b_guess]))
        output["objectives"].append(lasso_obj(b_guess))
        output["steps"].append(norm(b_old - b_guess, ord=np.inf))
        
        if norm(b_old - b_guess, ord=np.inf) < eps:
            output["status"] = "convergence"             
            return output  
                               
    output["status"] = "maxiter exceed"    
    
    return output


In [61]:
output

{'steps': [], 'estimate': [], 'objectives': [], 'status': []}

In [68]:
output["estimate"] = []


In [69]:
output["estimate"].append(3)

In [70]:
output["estimate"].append(2)

In [71]:
output["estimate"]

[3, 2]

In [81]:
from sklearn.linear_model import Lasso


In [98]:
lasso_f = Lasso(alpha=0.002, fit_intercept=True, normalize=True)

In [99]:
lasso_f.fit(X=X[:, 1:], y=Y)

Lasso(alpha=0.002, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

array([[ 0.8612071 ,  1.53429736,  0.7382611 , ...,  1.27414983,
         0.7382611 ,  0.7382611 ],
       [ 0.39201916,  1.06510942,  0.26907317, ...,  0.8049619 ,
         0.26907317,  0.26907317],
       [ 0.06852952,  0.74161978, -0.05441647, ...,  0.48147226,
        -0.05441647, -0.05441647],
       ...,
       [-0.23817198,  0.43491828, -0.36111797, ...,  0.17477076,
        -0.36111797, -0.36111797],
       [ 0.122946  ,  0.79603626,  0.        , ...,  0.53588873,
        -0.        , -0.        ],
       [ 0.122946  ,  0.79603626,  0.        , ...,  0.53588873,
        -0.        , -0.        ]])

In [94]:
b

array([[-7.38261102e-01],
       [-2.69073167e-01],
       [ 5.44164741e-02],
       [ 1.71246333e+00],
       [-8.82607687e-01],
       [ 1.43680815e-03],
       [ 1.23833596e+00],
       [-1.42979443e+00],
       [ 6.60423593e-01],
       [ 0.00000000e+00],
       [-1.10903242e+00],
       [ 9.84836443e-01],
       [-4.32414656e-01],
       [ 0.00000000e+00],
       [ 1.19982350e-01],
       [ 0.00000000e+00],
       [-5.41328998e-01],
       [-5.78044764e-01],
       [ 9.73787163e-01],
       [ 0.00000000e+00],
       [-6.03944706e-01],
       [-1.22482499e+00],
       [ 1.02199934e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 7.83442417e-01],
       [ 0.00000000e+00],
       [-8.96699490e-01],
       [ 7.17555007e-02],
       [ 8.27016321e-01],
       [-2.08164141e-01],
       [-3.41325832e-01],
       [ 7.47714441e-01],
       [-2.62963130e+00],
       [-1.05621217e+00],
       [-1.10043107e+00],
       [ 0.00000000e+00],
       [ 4.76619636e-01],
       [-8.8